<a href="https://colab.research.google.com/github/alphakilo11/Python/blob/main/Fantasy%20Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# inspired by the simple Fantasy Game Inventory exercise in Al Sweigarts Book 'Automating the Boring Stuff with Python' i decided to create a Fantasy Game

"""
Überlegungen

Ein besonderer Faktor sollen zufällig generierte Inhalte sein.

Ich bin sehr unentschlossen in Bezug auf das Kampfsystem; meine Idee wäre dieses möglichst realistisch zu gestalten, doch dazu fehlen mir die Grundlagen.
Was bewirkt Kraft im Bezug auf einen Schwerthieb? Ist nicht Geschwindigkeit wichtiger?
Geschicklichkeit ist vermutlich wichtig um ein Ziel mit einer Waffe zu treffen
Verletzlichkeit hängt wohl stark mit der Anatomie zusammen; ein Regenwurm kann in der Hälfte durchtrennt werden und lebt weiter, während ein gezielter Stich bereits ausreicht um einen Menschen zu töten.
Viele Verletzungen für nicht unmittelbar zur Kampfunfähigkeit besonders wenn die betroffenen unter Adrenalin stehen.

Die Spielmechanik sollte einen gewissen Glücksfaktor enthalten um all jene Faktoren, welche nicht "simuliert" werden zu würdigen.

Moral
Ein leicht bewaffneter Bauer wird kaum gegen einen Ritter in Plattenrüstung kämpfen, solange er noch bei Sinnen ist.

Interface
Unabhängig von der gewählten Sprache, müssen die Steuerungstasten gleich bleiben.
Eventuell sollten nach jeder falschen Eingabe die Auswahlmöglichkeiten neu angezeigt werden.
"""

# https://2e.aonprd.com/
#ISSUE the input-processing is completely independent from the text dictionaries - while the input check uses the dictionaries as source. This is sub-optimal.
# TODO englische Übersetzung



'\nÜberlegungen\n\nEin besonderer Faktor sollen zufällig generierte Inhalte sein.\n\nIch bin sehr unentschlossen in Bezug auf das Kampfsystem; meine Idee wäre dieses möglichst realistisch zu gestalten, doch dazu fehlen mir die Grundlagen.\nWas bewirkt Kraft im Bezug auf einen Schwerthieb? Ist nicht Geschwindigkeit wichtiger?\nGeschicklichkeit ist vermutlich wichtig um ein Ziel mit einer Waffe zu treffen\nVerletzlichkeit hängt wohl stark mit der Anatomie zusammen; ein Regenwurm kann in der Hälfte durchtrennt werden und lebt weiter, während ein gezielter Stich bereits ausreicht um einen Menschen zu töten.\nViele Verletzungen für nicht unmittelbar zur Kampfunfähigkeit besonders wenn die betroffenen unter Adrenalin stehen.\n\nDie Spielmechanik sollte einen gewissen Glücksfaktor enthalten um all jene Faktoren, welche nicht "simuliert" werden zu würdigen.\n\nMoral\nEin leicht bewaffneter Bauer wird kaum gegen einen Ritter in Plattenrüstung kämpfen, solange er noch bei Sinnen ist.\n\nInterfac

In [14]:
def displayInventory(inventory_dict):
  leftWidth = 10
  rightWidth = 10
  print("Inventory:")
  total_items = 0
  for i,k in inventory.items():
    print(i.ljust(leftWidth), str(k).rjust(rightWidth))
    total_items += k
  print("Total number of items:", total_items)

def addToInventory(inventory, addedItems):
    for i in addedItems:
        if i in inventory:
            old_value = inventory.get(i)
            new_value = old_value + 1
            inventory[i] = new_value
        else:
            inventory.setdefault(i, 1)
    print(len(addedItems), "items", addedItems, "added.")
    return inventory

""" not working
def LanguageSelect(): #TODO update 
  while True:
    lang = input("Select language: 'e' for englisch, 'g' für Deutsch.")
    lang = lang.lower()
    if lang != 'e' and lang != 'g':
      print("Es sind keine andere Eingabe möglich. / No other entry possible") # text_actual is not available it this time
    else:
      if lang == "g":
        text_actual = text["german"]
      if lang == "e":
        text_actual = text["english"]
      break
"""
# User Interface
def UserInput(options_dict):
  """ Displays passed descriptions and keys, gets user input and checks if user input matches the keys."""
# display
  keys_lower = []
  for description, key in options_dict.items():
    print(key, description)
    keys_lower.append(key.lower())
# input and check  
  while True:
    eingabe = input(text[lang]["user_input"])
    if eingabe.lower() in keys_lower:
      return eingabe.lower()
    else:
      print(text[lang]["input_not_available"])

def menu_main():
  from IPython.display import clear_output
  input(text[lang]["press_continue"])
  clear_output(wait=True)
  eingabe = UserInput(text[lang]["main_menu_prompt"])
  clear_output(wait=True)

  if eingabe == "n": # new game
    Character = copy.deepcopy(defaultCharacter) # reset combatants
    Hostile = copy.deepcopy(defaultHostile)
    combat((Character, Hostile))

  if eingabe == "e":
    menu_settings()
  
  if eingabe == "c": #credits
    print(text[lang]["author"], "Alexander Krendl")

  if eingabe == "b": #quit game
    print(text[lang]["quit"])
    return
  
  menu_main()


def menu_settings():
  eingabe = UserInput(text[lang]["settings_prompt"])
  if eingabe == "s": # show values (like dice roll results)
    global show_values
    if show_values:
      show_values = False
      print(text[lang]["values_off"])
    else:
      show_values = True
      print(text[lang]["values_on"])
    return #TODO returns to main menu - should be settings menu
  if eingabe == "z":
    return

#game mechanics

def rollDice():
  import random
  return random.randint(1,100)

def attack(attacker, defender, weapon): #should I include a weapon used or make this part of the character? Looking at monsters that have fangs and talons it might be better to include that here.
  """ calculates an attack """
  #TODO include modifiers
  #TODO include damage? (or use seperate function?)
  attackroll = rollDice() + weapon[0]
  defendroll = rollDice()
  if show_values:
    print(text[lang]["dice_roll"], ":", attacker["Stats"]["name"], attackroll - weapon[0], "+", weapon[0], "=", attackroll, defender["Stats"]["name"], defendroll)
  if defendroll > attackroll:
    return "miss"
  else: # a tie is counted as hit (thereby favoring the attacker)
    return "hit"

def damage(attacker, defender, weapon):
  defender["Stats"]["Hitpoints"] = defender["Stats"]["Hitpoints"] -5 # damage placeholder
  print(defender["Stats"]["name"], "Hitpoints:", defender["Stats"]["Hitpoints"])

def combat(combatants):
  #TODO initiative
  round = 1
  while True:
    
    if round % 2 != 0:
      attacker = combatants[0]
      defender = combatants[1]
    else:
      attacker = combatants[1]
      defender = combatants[0]
    if attack(attacker, defender, attacker["Stats"]["active_weapon"]) == "hit":
      damage(attacker, defender, attacker["Stats"]["active_weapon"])
      if defender["Stats"]["Hitpoints"] <= 0:
        print(defender["Stats"]["name"], text[lang]["lost"], ".")
        break
    else:
      print(attacker["Stats"]["name"], text[lang]["missed"], defender["Stats"]["name"])
    round = round + 1


In [9]:
# Gamedata
weapons = {"Short Sword": [5, 3]} # At the end of the battle, you can recover half your expended ammunition by taking a minute to search the battlefield.
text = {
    "german": {
          "main_menu_prompt": {"Fortfahren [NOT IMPLEMENTED YET]": "f", "Neues Spiel starten [NOT IMPLEMENTED YET]": "n", "Spielstand laden [NOT IMPLEMENTED YET]": "l", "Einstellungen": "e", "Credits": "c", "Beenden": "b"},
         "input_not_available": "Diese Eingabe ist im Moment nicht möglich.",
         "user_input": "Bitte wählen Sie aus den oben angeführten Optionen:",
         "settings_prompt": {"Spielmechanik-Werte anzeigen": "s", "Zurück": "z"},
         "quit": "Spiel wird beendet.",
         "values_off": "Spielmechanik-Werte werden nicht angezeigt.",
         "values_on": "Spielmechanik-Werte werden angezeigt.",
         "author": "Author:",
          "dice_roll": "Alea iacta est",
         "press_continue": "Beliebige Taste drücken um fortzusetzen",
         "missed": "verfehlt",
         "lost": "wurde besiegt"
         },
    "english": {
             "main_menu_prompt": "What is your desire? 'i' Inventory 'e' explore 'q' quit", "input_not_available": "Unable to process input."}
        }
defaultCharacter = {
"Inventory": {'rope': 1, 'torch': 6, 'gold coin': 42, 'dagger': 1, 'arrow': 12},
"Attributes": {"Kraft": 10, "Geschicklichkeit": 10, "Ausdauer": 10},
"Stats": {"name": "Hulk", "Level": 1, "XP": 0, "Hitpoints": 10, "active_weapon": weapons["Short Sword"]},
"Conditions": []}

defaultHostile = {
"Inventory": {'rope': 1, 'torch': 6, 'gold coin': 42, 'dagger': 1, 'arrow': 12},
"Attributes": {"Kraft": 10, "Geschicklichkeit": 10, "Ausdauer": 10},
"Stats": {"name": "Skeletor", "Level": 1, "XP": 0, "Hitpoints": 10, "active_weapon": weapons["Short Sword"]},
"Conditions": []}


In [15]:
# base setup
import copy
Character = copy.deepcopy(defaultCharacter)
Hostile = copy.deepcopy(defaultHostile)
lang = "german"
show_values = True

# main instance
menu_main()


Spiel wird beendet.


In [5]:
#displayInventory(Character["Inventory"])
#dragonLoot = ['gold coin', 'gold coin', 'dagger', 'gold coin', 'ruby']
#
#addToInventory(inventory, dragonLoot)
#displayInventory(Character["Inventory"])

In [6]:
# https://www.dndcombat.com/dndcombat/Welcome.do#
# if Class == Light: add Dex modifier, Medium: add Dex modifier (max 2)
categories = ["Name", "Class", "Cost", "Armor Class (AC)", "Strength", "Stealth", "Weight"]
item = ["Plate", "Heavy Armor", 1500, 18, 15, "Disadvantage", 65]
item_dict = {}
for i, j in zip(categories, item):
    item_dict.setdefault(i, j)

In [10]:
defaultHostile["Stats"]["Hitpoints"]

10